In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report

import pymorphy2
morph = pymorphy2.MorphAnalyzer()
from sklearn.pipeline import Pipeline
#from sklearn.metrics.scorer import make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

In [2]:
#with open ('dataset.txt',encoding='UTF-8') as file:
#    lines = file.readlines()
#    lines = [line.rstrip() for line in lines]

In [3]:
data=pd.read_excel('data.xlsx', dtype=object)
#for line in lines:
#    tags=line.split(' ',1)[0]
#    for tag in tags.split(','):
#        data.append([tag.replace('__label__',''),line.split(' ',1)[1]])

In [4]:
%%time
def clean(txt):
    lst = re.sub('[^А-Яа-яa-zA-Z]+', ' ', str(txt)).lower()

    return lst

data['TEXT']=data['TEXT'].apply(clean)

Wall time: 778 ms


In [5]:
data.head()

,Unnamed: 0,TARGET,TEXT
0,0,INSULT,скотина что сказать
1,7,INSULT,заколоть этого плешивого урода что бы крякнул ...
2,8,THREAT,заколоть этого плешивого урода что бы крякнул ...
3,25,INSULT,долбоебы это фэйк
4,26,INSULT,пиздаболы сделали снимок придумали историю и п...


In [6]:
data.TARGET.value_counts()

INSULT       36826
NORMAL       20000
THREAT       12027
OBSCENITY     4261
Name: TARGET, dtype: int64

In [7]:
#data=pd.DataFrame(data)
#data.columns=['TARGET','TEXT']

In [8]:
#data.TARGET.value_counts()

In [9]:
#data=pd.concat([data[data['TARGET'].isin(['INSULT','THREAT','OBSCENITY'])],data[data['TARGET']=='NORMAL'].head(20000)])

In [10]:
#data.TARGET.value_counts()

### Нормализация

In [11]:
def norm(txt):
    lst = re.sub('[^А-Яа-яa-zA-Z]+', ' ', txt).lower() #оставляет только А-я, A-z
    lst = lst.split(' ')

    words = []
    
    for word in lst:
        p = morph.parse(word)[0]
        words.append(p.normal_form)

    return ' '.join(words)

In [12]:
#ctest= CountVectorizer()
#ftest=TfidfVectorizer(use_idf=True)

In [13]:
# analyzer = 'word','char'

# max_df
# min_df
# max_features

In [14]:
data.TEXT.values[0]

'скотина что сказать'

In [15]:
norm(data.TEXT.values[0])

'скотина что сказать'

In [16]:
# def my_custom_loss_func(y_true, y_pred):
#     error=np.abs((y_true - y_pred)/y_true)
#     return np.mean(error)*100

# my_scorer = make_scorer(my_custom_loss_func,greater_is_better=False)

In [17]:
#np.linspace(1000,10000,4)

In [18]:
pipe = Pipeline([
    ('vec', TfidfVectorizer()),    
    ('sgd', SGDClassifier())
    ])


cv_params = dict([
    ('vec__analyzer', ['word','char']),
    ('vec__ngram_range',[(1,1),(1,2)]  ),
    ('vec__max_features',np.linspace(1000,10000,4).astype(int)) ,
    ('vec__sublinear_tf',[True,False]  ),
    ('sgd__alpha', 10.0**np.arange(-4,1)),
    ('sgd__loss', ['modified_huber']),
])

In [19]:
np.linspace(2000,10000,5) 

array([ 2000.,  4000.,  6000.,  8000., 10000.])

In [20]:
X_train, X_test, y_train, y_test = train_test_split(data.TEXT.values, data.TARGET.values, test_size=0.2, random_state=42)

In [21]:
# model = GridSearchCV(pipe, cv_params, verbose=1, n_jobs=-1,scoring='f1_weighted', cv = 3) #запустить его, если будет время 
model = RandomizedSearchCV(pipe, cv_params, verbose=1, n_jobs=-1,scoring='f1_weighted', cv = 3, n_iter=10)
# лучше ставить model = RandomizedSearchCV(pipe, cv_params, verbose=1, n_jobs=-1,scoring='f1_weighted', cv >5, n_iter> 100)

model.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('vec', TfidfVectorizer()),
                                             ('sgd', SGDClassifier())]),
                   n_jobs=-1,
                   param_distributions={'sgd__alpha': array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00]),
                                        'sgd__loss': ['modified_huber'],
                                        'vec__analyzer': ['word', 'char'],
                                        'vec__max_features': array([ 1000,  4000,  7000, 10000]),
                                        'vec__ngram_range': [(1, 1), (1, 2)],
                                        'vec__sublinear_tf': [True, False]},
                   scoring='f1_weighted', verbose=1)

In [22]:
model.best_estimator_

Pipeline(steps=[('vec', TfidfVectorizer(max_features=10000)),
                ('sgd', SGDClassifier(loss='modified_huber'))])

In [23]:
model.best_params_

{'vec__sublinear_tf': False,
 'vec__ngram_range': (1, 1),
 'vec__max_features': 10000,
 'vec__analyzer': 'word',
 'sgd__loss': 'modified_huber',
 'sgd__alpha': 0.0001}

In [24]:
model.best_score_

0.754945341808874

In [25]:
model.best_estimator_[1].coef_.shape

(4, 10000)

In [26]:
model

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('vec', TfidfVectorizer()),
                                             ('sgd', SGDClassifier())]),
                   n_jobs=-1,
                   param_distributions={'sgd__alpha': array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00]),
                                        'sgd__loss': ['modified_huber'],
                                        'vec__analyzer': ['word', 'char'],
                                        'vec__max_features': array([ 1000,  4000,  7000, 10000]),
                                        'vec__ngram_range': [(1, 1), (1, 2)],
                                        'vec__sublinear_tf': [True, False]},
                   scoring='f1_weighted', verbose=1)

In [27]:
model.best_estimator_[0]

TfidfVectorizer(max_features=10000)

In [28]:
y_pred = model.predict(X_test)

In [29]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      INSULT       0.77      0.80      0.79      7342
      NORMAL       0.82      0.89      0.86      3909
   OBSCENITY       0.65      0.47      0.54       864
      THREAT       0.66      0.58      0.62      2508

    accuracy                           0.77     14623
   macro avg       0.73      0.68      0.70     14623
weighted avg       0.76      0.77      0.76     14623



In [30]:
i=0
feats= model.best_estimator_.steps[0][1].get_feature_names()
coefs= model.best_estimator_.steps[1][1].coef_
res= pd.DataFrame()
for coef in coefs:
    buf_df=pd.DataFrame()
    buf_df['feat']=feats
    buf_df['coef']=coef
    buf_df = buf_df.sort_values(by=['coef'],ascending=False).head(20)
    res[model.best_estimator_.steps[1][1].classes_[i]]=buf_df.feat.values
    i=i+1
    
#     

In [31]:
res

,INSULT,NORMAL,OBSCENITY,THREAT
0,долбо,здоровья,сосать,расстрелять
1,дебилы,спасибо,трахать,убивать
2,пидоры,здравствуйте,трахаться,расстрел
3,дура,праздником,вдул,расстреливать
4,блядь,цена,насосала,убить
5,долбоебы,красота,трахнуть,растрелять
6,пидорасы,золотые,ебут,кол
7,долба,очень,трахнул,стрелять
8,долбоеб,счастья,трахал,отстреливать
9,долбоебов,память,сос,бить
